In [1]:
from secml.adv.attacks.poisoning import CAttackPoisoningLogisticRegression
from secml.ml.classifiers import CClassifierLogistic
from secml.ml.features.normalization import CNormalizerMinMax
from secml.data import CDataset
from secml.ml.peval.metrics import CMetricAccuracy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from secml.ml.classifiers import CClassifierRidge
from secml.adv.attacks.poisoning import CAttackPoisoningRidge


In [2]:
data = pd.read_csv("darknet_data.csv").values
datax = data[:, 1:]
datay = data[:, 0]
random_state = 2
x_train, x_test, y_train, y_test = train_test_split(
    datax, datay, test_size=250, train_size=1000, random_state=random_state)
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test, test_size=125, train_size=125, random_state=random_state)


In [4]:
training_data = CDataset(x_train, y_train)
validation_data = CDataset(x_val, y_val)
test_data = CDataset(x_test,y_test)
metric = CMetricAccuracy()
clf=CClassifierRidge(max_iter=100)

In [5]:
clf.fit(training_data.X, training_data.Y)
print("Training of classifier complete!")
# Compute predictions on a test set
y_pred = clf.predict(test_data.X)

# Bounds of the attack space. Can be set to `None` for unbounded
lb, ub = validation_data.X.min(), validation_data.X.max()
# Number of poisoning points to generate
n_poisoning_points = 500

solver_params = {
    'eta': 0.05,
    'eta_min': 0.05,
    'eta_max': None,
    'max_iter': 100,
    'eps': 1e-6
}


Training of classifier complete!


In [6]:
pois_attack = CAttackPoisoningRidge(classifier=clf,
                                    training_data=training_data,
                                    val=validation_data,
                                    lb=0, ub=1,
                                    solver_params=solver_params,
                                    random_seed=random_state,
                                    init_type='random')

pois_attack.n_points = n_poisoning_points


In [7]:
print("Attack started...")
pois_y_pred, pois_scores, pois_ds, f_opt = pois_attack.run(
    test_data.X, test_data.Y)
print("Attack complete!")

Attack started...
Attack complete!


In [8]:
acc = metric.performance_score(y_true=test_data.Y, y_pred=y_pred)
# Evaluate the accuracy after the poisoning attack
pois_acc = metric.performance_score(y_true=test_data.Y, y_pred=pois_y_pred)

print("Original accuracy on test set: {:.2%}".format(acc))
print(
    "Accuracy after non adaptive attack on test set: {:.2%}".format(pois_acc))


Original accuracy on test set: 82.40%
Accuracy after non adaptive attack on test set: 70.40%


In [14]:
pd.DataFrame(pois_ds.X).to_csv("rx_new.csv",index=False)
temp=pd.read_csv("rx_new.csv").values
temp=temp.reshape(-1,76)
pd.DataFrame(temp).to_csv("rx_new.csv",index=False)
pd.DataFrame(pois_ds.Y).to_csv("ry_new.csv",index=False)